In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
# hyperparameters
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-3
eval_liters = 250
